# Entry & Exit Signal

- Create separate notebook as 'poc.ipynb' notebook is too bloated.
- Create abstract class


In [1]:
import pandas as pd
from abc import ABC, abstractmethod
from datetime import datetime, date
import re
from collections import deque, Counter
from decimal import Decimal
import numpy as np
import yfinance as yf
from pydantic import BaseModel, Field, computed_field, field_validator
import math
from pathlib import Path
from zoneinfo import ZoneInfo
import calendar
import itertools

%load_ext autoreload
%autoreload 2
%matplotlib inline

from src.utils import utils
from config.variables import EntryType, COINT_CORR_FN, STRUCT_MAPPING, EXIT_PRICE_MAPPING
from src.strategy.base import StockTrade

# Table of Contents

1. [GenTrades](#gentrades)


In [14]:
trade = StockTrade(
    ticker="AAPL",
    entry_datetime="2025-04-01",
    entry_action="buy",
    entry_lots=10,
    entry_price=200,
)
trade1 = StockTrade(
    ticker="AAPL",
    entry_datetime="2025-04-02",
    entry_action="buy",
    entry_lots=10,
    entry_price=195,
)

open_trades = deque([trade, trade1])
class_name = STRUCT_MAPPING.get("multiple")
entry_method = utils.get_class_instance(
    class_name, "./src/strategy/base/entry_struct.py", num_lots=20
)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-4-4", "buy", 192)
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 4, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [19]:
utils.display_open_trades(open_trades)

open_trades : 
[
   ticker: 'AAPL', ent_dt: '2025-04-01', ent_act: 'buy', ent_lots: 10, ent_price: 200, ex_dt: 'None', ex_act: 'None', ex_lots: 0, ex_price: None
   ticker: 'AAPL', ent_dt: '2025-04-02', ent_act: 'buy', ent_lots: 10, ent_price: 195, ex_dt: 'None', ex_act: 'None', ex_lots: 0, ex_price: None
   ticker: 'AAPL', ent_dt: '2025-04-04', ent_act: 'buy', ent_lots: 20, ent_price: 192, ex_dt: 'None', ex_act: 'None', ex_lots: 0, ex_price: None
]



In [140]:
open_trades = deque()
class_name = STRUCT_MAPPING.get("multiple_half")
entry_method = utils.get_class_instance(
    class_name, "./src/strategy/base/entry_struct.py", num_lots=20
)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-01", "sell", 192)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-02", "sell", 196)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-03", "sell", 193)
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='sell', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='sell', entry_lots=Decimal('10'), entry_price=Decimal('196'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 3, 0, 0), entry_action='sell', entry_lots=Decimal('5'), entry_price=Decimal('193'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [50]:
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 4, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [51]:
class_name = STRUCT_MAPPING.get("half_lifo")
exit_method = utils.get_class_instance(class_name, "./src/strategy/base/exit_struct.py")
open_trades, completed_trades = exit_method.close_pos(
    open_trades, "2025-04-10", "buy", 180
)
open_trades

Validation Error : 1 validation error for StockTrade
exit_action
  Value error, Entry action cannot be the same as exit action except when both are 'wait'. [type=value_error, input_value='buy', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 4, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [131]:
open_trades = deque()
class_name = STRUCT_MAPPING.get("multiple_half")
entry_method = utils.get_class_instance(
    class_name, "./src/strategy/base/entry_struct.py", num_lots=20
)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-01", "buy", 192)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-02", "buy", 196)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-03", "buy", 193)
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('196'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 3, 0, 0), entry_action='buy', entry_lots=Decimal('5'), entry_price=Decimal('193'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [141]:
class_name = EXIT_PRICE_MAPPING.get("percent_loss")
stop_price = utils.get_class_instance(
    class_name, "./src/strategy/base/cal_exit_price.py", percent_loss=0.1
)
stop_price.cal_exit_price(open_trades)

Decimal('212.61')

In [30]:
pos = "long"
trade1 = (10, 141.87)
trade2 = (10, 141.97)
trade3 = (10, 143.25)
percent_loss = 0.001
trades = [
    trade1,
    trade2,
    trade3,
]
total_lots = sum(trade[0] for trade in trades)
total_invest = sum(trade[0] * trade[1] for trade in trades)
# stop_loss = [trade[1] * (1-percent_loss) if pos == "long" else trade[1] * (1+percent_loss) for trade in trades]
# stop_loss = [Decimal(str(round(loss, 2))) for loss in stop_loss]
# stop_loss

# (10 * 200 + 10 * 195 + 20 * 192 ) * (1-percent_loss)/ 40

stop_price = (
    total_invest * (1 - percent_loss) / total_lots
    if pos == "long"
    else total_invest * (1 + percent_loss) / total_lots
)
stop_price

142.22096999999997

In [32]:
142.9 * (1-percent_loss)

142.7571

# [GenTrades](#table-of-contents)


In [2]:
date = "2025-04-01"
model = "ziweichen_coint_5"
ticker_pair = "NFLX_EA"

results_dir = f"./data/results/{date}"
model_dir = f"{results_dir}/{model}"
pa_dir = f"{model_dir}/price_actions"

In [3]:
df = utils.load_csv(f"{pa_dir}/{ticker_pair}.csv")
df

,date,coint_corr_ticker,open,high,low,close,volume,ticker,day_name,av_rating,median_rating_excl
0,2025-03-03,EA,130.38,131.6,128.87,129.23,5209500,NFLX,Monday,3.0,3.0
1,2025-03-04,EA,129.52,133.35,129.24,131.82,4291000,NFLX,Tuesday,3.0,3.0
2,2025-03-05,EA,131.6,134.51,131.31,134.05,5218000,NFLX,Wednesday,3.25,4.0
3,2025-03-06,EA,133.65,137.53,133.37,136.79,3676700,NFLX,Thursday,4.5,5.0
4,2025-03-07,EA,136.19,140.42,136.0,140.04,3356800,NFLX,Friday,3.0,3.0
5,2025-03-10,EA,139.75,142.97,139.22,140.43,5440900,NFLX,Monday,5.0,5.0
6,2025-03-11,EA,140.9,140.9,136.9,137.88,3837200,NFLX,Tuesday,3.5,4.0
7,2025-03-12,EA,137.0,138.15,135.73,136.12,3177500,NFLX,Wednesday,3.0,3.0
8,2025-03-13,EA,135.79,138.26,135.23,137.72,3165100,NFLX,Thursday,3.1666666666666665,3.5
9,2025-03-14,EA,138.15,139.23,137.87,138.71,3065500,NFLX,Friday,3.0,3.0


In [91]:
# entry_type = "long_only"
# entry_type = "short_only"
entry_type = "long_or_short"

entry = utils.get_class_instance(
    "SentiEntry",
    "./src/strategy/entry/senti_entry.py",
    entry_type=entry_type,
)
exit = utils.get_class_instance(
    "SentiExit",
    "./src/strategy/exit/senti_exit.py",
    entry_type=entry_type,
)

df_signal = entry.gen_entry_signal(df)
df_signal = exit.gen_exit_signal(df_signal)
df_signal

,date,coint_corr_ticker,open,high,low,close,volume,ticker,day_name,av_rating,median_rating_excl,entry_signal,exit_signal
0,2025-03-03,EA,130.38,131.6,128.87,129.23,5209500,NFLX,Monday,3.0,3,wait,wait
1,2025-03-04,EA,129.52,133.35,129.24,131.82,4291000,NFLX,Tuesday,3.0,3,wait,wait
2,2025-03-05,EA,131.6,134.51,131.31,134.05,5218000,NFLX,Wednesday,3.25,4,buy,buy
3,2025-03-06,EA,133.65,137.53,133.37,136.79,3676700,NFLX,Thursday,4.5,5,buy,buy
4,2025-03-07,EA,136.19,140.42,136.0,140.04,3356800,NFLX,Friday,3.0,3,wait,wait
5,2025-03-10,EA,139.75,142.97,139.22,140.43,5440900,NFLX,Monday,5.0,5,buy,buy
6,2025-03-11,EA,140.9,140.9,136.9,137.88,3837200,NFLX,Tuesday,3.5,4,buy,buy
7,2025-03-12,EA,137.0,138.15,135.73,136.12,3177500,NFLX,Wednesday,3.0,3,wait,wait
8,2025-03-13,EA,135.79,138.26,135.23,137.72,3165100,NFLX,Thursday,3.1666666666666665,3,wait,wait
9,2025-03-14,EA,138.15,139.23,137.87,138.71,3065500,NFLX,Friday,3.0,3,wait,wait


In [100]:
trades = utils.get_class_instance(
    "SentiTrades",
    "./src/strategy/trade/senti_trades.py",
    entry_struct="single",
    exit_struct="half_lifo",
    num_lots=10,
    percent_loss=0.01,
    exit_method="percent_loss",
    monitor_close=False,
)

df_trades, df_signal = trades.gen_trades(df_signal)
df_trades

idx : 0
dt : 2025-03-03 16:00:00-05:00
close : 129.23
ent_sig : wait
ex_sig : wait
net_pos : 0
net_pos after update : 0
len(self.open_trades) : 0
open_trades : []

idx : 1
dt : 2025-03-04 16:00:00-05:00
close : 131.82
ent_sig : wait
ex_sig : wait
net_pos : 0
net_pos after update : 0
len(self.open_trades) : 0
open_trades : []

idx : 2
dt : 2025-03-05 16:00:00-05:00
close : 134.05
ent_sig : buy
ex_sig : buy
net_pos : 0
net_pos after update : 10
len(self.open_trades) : 1
open_trades : 
[
   {
      ticker: 'EA', ent_dt: '2025-03-05', ent_act: 'buy', ent_lots: 10, ent_price: 134.05, ex_dt: None, ex_act: None, ex_lots: 0, ex_price: None
   },
]

idx : 3
dt : 2025-03-06 16:00:00-05:00
close : 136.79
ent_sig : buy
ex_sig : buy
net_pos : 10
stop_price [long] : 132.71 -> monitor low [current: 133.37]
net_pos after update : 10
len(self.open_trades) : 1
open_trades : 
[
   {
      ticker: 'EA', ent_dt: '2025-03-05', ent_act: 'buy', ent_lots: 10, ent_price: 134.05, ex_dt: None, ex_act: None, ex_lo

,news_ticker,ticker,entry_datetime,entry_action,entry_lots,entry_price,exit_datetime,exit_action,exit_lots,exit_price,days_held,profit_loss,percent_ret,daily_ret,win
0,NFLX,EA,2025-03-05 16:00:00-05:00,buy,10,134.05,2025-03-21 16:00:00-04:00,sell,10,144.3,15,10.25,0.076464,0.004924,1
1,NFLX,EA,2025-03-21 16:00:00-04:00,sell,10,144.3,2025-03-24 16:00:00-04:00,buy,10,145.74,3,-1.44,-0.009979,-0.003337,0
2,NFLX,EA,2025-03-24 16:00:00-04:00,buy,10,144.55,2025-03-27 16:00:00-04:00,sell,10,145.3,3,0.75,0.005189,0.001727,1
3,NFLX,EA,2025-03-27 16:00:00-04:00,sell,10,145.3,2025-03-31 16:00:00-04:00,buy,10,144.52,4,0.78,0.005368,0.001339,1


In [63]:
pos = "long"
percent_loss = 0.01
trades = [
    (10, 134.05),
    (5, 136.79),
    (3, 140.43),
    (2, 137.88),
    # (10, 144.5),
]
total_lots = sum(trade[0] for trade in trades)
total_invest = sum(trade[0] * trade[1] for trade in trades)

stop_price = (
    total_invest * (1 - percent_loss) / total_lots
    if pos == "long"
    else total_invest * (1 + percent_loss) / total_lots
)
stop_price

134.71425

In [50]:
137.88 * (1-percent_loss)

137.74212

In [53]:
for ent_type, ent, ex in itertools.product(["long_only", "short_only"], ["multiple", "multiple_half"], ["fifo", "lifo", "half_fifo", "half_lifo", "take_all"]):
    print(ent_type, ent, ex)

long_only multiple fifo
long_only multiple lifo
long_only multiple half_fifo
long_only multiple half_lifo
long_only multiple take_all
long_only multiple_half fifo
long_only multiple_half lifo
long_only multiple_half half_fifo
long_only multiple_half half_lifo
long_only multiple_half take_all
short_only multiple fifo
short_only multiple lifo
short_only multiple half_fifo
short_only multiple half_lifo
short_only multiple take_all
short_only multiple_half fifo
short_only multiple_half lifo
short_only multiple_half half_fifo
short_only multiple_half half_lifo
short_only multiple_half take_all


In [54]:
a = list(itertools.product(["long_only", "short_only"], ["multiple", "multiple_half"], ["fifo", "lifo", "half_fifo", "half_lifo", "take_all"])) + list(itertools.product(["long_only", "short_only"], ["single"], ["fifo", "half_fifo"])) + list (itertools.product(["long_or_short"], ["multiple", "multiple_half", "single"], ["take_all"]))
a

[('long_only', 'multiple', 'fifo'),
 ('long_only', 'multiple', 'lifo'),
 ('long_only', 'multiple', 'half_fifo'),
 ('long_only', 'multiple', 'half_lifo'),
 ('long_only', 'multiple', 'take_all'),
 ('long_only', 'multiple_half', 'fifo'),
 ('long_only', 'multiple_half', 'lifo'),
 ('long_only', 'multiple_half', 'half_fifo'),
 ('long_only', 'multiple_half', 'half_lifo'),
 ('long_only', 'multiple_half', 'take_all'),
 ('short_only', 'multiple', 'fifo'),
 ('short_only', 'multiple', 'lifo'),
 ('short_only', 'multiple', 'half_fifo'),
 ('short_only', 'multiple', 'half_lifo'),
 ('short_only', 'multiple', 'take_all'),
 ('short_only', 'multiple_half', 'fifo'),
 ('short_only', 'multiple_half', 'lifo'),
 ('short_only', 'multiple_half', 'half_fifo'),
 ('short_only', 'multiple_half', 'half_lifo'),
 ('short_only', 'multiple_half', 'take_all'),
 ('long_only', 'single', 'fifo'),
 ('long_only', 'single', 'half_fifo'),
 ('short_only', 'single', 'fifo'),
 ('short_only', 'single', 'half_fifo'),
 ('long_or_short

In [55]:
len(a)

27